# Introduction

<img src='./images/equity_equality.jpg'>

<center> Equality vs. equity <br> <i> <a href="interactioninstitute.org">Interaction Institute for Social Change</a> // Angus Maguire</i> </center>

**SUMMARY**

The purpose of this report is to analyze health inequities in the US by county and how those inequities influence the degree of impact of the COVID-19 pandemic.

____

**STAKEHOLDERS**

Federal, state, and local governments and epidemiologists can use this analysis to shape public health policy and pandemic response plans. The goal is to learn from COVID-19 to prevent the loss that occurred in 2020-2021 from happening again.

____

**DATA SOURCES**

- [CDC’s Social Vulnerability Index](https://www.atsdr.cdc.gov/placeandhealth/svi/documentation/SVI_documentation_2018.html)

- [CDC's Vaccine Hesitancy](https://data.cdc.gov/Vaccinations/Vaccine-Hesitancy-for-COVID-19-County-and-local-es/q9mh-h2tw)

- [The University of Wisconsin Population Health Institute's County Health Rankings](https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources) 

- [New York Times COVID Case and Death Counts](https://github.com/nytimes/covid-19-data)


____

**METHODS**

Classification model which predicts a given county’s degree of impact of the pandemic (high impact, average impact, low impact). Feature importance and coefficients will be analyzed to recommend what features put communities at greater risk in order to design policies to address the underlying issues.

____

**LOAD PACKAGES**

In [1]:
# style notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [2]:
#import standard packages
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

#import viz packages
import matplotlib.pyplot as plt
%matplotlib inline
#plt.rcParams['figure.figsize'] = (15,5) #setting figures to timeseries-friendly
import plotly.express as px
import plotly.graph_objects as go

#import time series tools from statsmodels
import statsmodels.tsa.api as tsa
import statsmodels
print(f'Statsmodels version = {statsmodels.__version__}')

Statsmodels version = 0.12.0


# Data Load

## Compress

Data files downloaded from data.cdc.gov, Univ of Wisconsin Public Health Institute, and the New York Times are too large to push to github in their raw form, so the first step is to compress the files.

**Social Vulnerability Index:**

In [3]:
svi = pd.read_csv('data/SVI2018_US_COUNTY.csv')
svi.head()

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,71,12398,439,-999,-999,-999,-999,-999,-999,3669,426,7083,25,9318,14,6280,495,1330,285.0,34397,145.0,755,209.5,67,37.1,7770,431,264,77.1,763,160,654,142,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,13.8,1.6,18.0,0.1,23.7,0.0,16.1,1.3,10.7,2.3,87.5,0.4,2.1,0.6,0.3,0.2,38.8,2.2,2.1,0.6,6.2,1.3,1.7,0.4,-999.0000,-999.0000,-999.0000,0.5922,-999.0000,-999.0000,0.4893,0.6826,0.5610,0.8383,2.5712,0.8758,0.9917,0.7740,1.7657,0.9268,0.0551,0.9869,0.5498,0.5788,0.4126,2.5832,0.5409,-999.0000,-999.0000,-999,-999,-999,0,-999,0,0,0,0,0,1,0,1,0,1,0,0,0,1,-999,4160,588,10.6,1.5,32290
1,1,ALABAMA,AL,Autauga,1001,"Autauga County, Alabama",594.443459,55200,0,23315,71,21115,383,8422,1137,1065,257,29372,2306,4204,475,8050,75,13369,32,10465,729,1586,319.9,13788,59.0,426,205.9,886,308.7,4279,469,299,142.3,1191,272,546,161,15.4,2.1,4.2,1.0,29372.0,2306.0,11.3,1.3,14.6,0.1,24.2,0.1,19.3,1.3,7.5,1.5,25.0,0.1,0.8,0.4,3.8,1.3,18.4,2.0,1.4,0.7,5.6,1.3,1.0,0.3,0.5401,0.2745,0.2860,0.4397,1.5403,0.3631,0.1850,0.7529,0.7905,0.3792,2.1076,0.5810,0.6336,0.5113,1.1449,0.5947,0.6017,0.7408,0.2964,0.4846,0.1525,2.2760,0.3741,7.0688,0.4354,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3875,508,7.1,0.9,37301
2,1,ALABAMA,AL,Blount,1009,"Blount County, Alabama",644.830460,57645,0,24222,55,20600,396,8220,992,909,193,22656,905,7861,727,10233,91,13468,53,8114,592,1437,267.2,7413,229.0,934,239.3,211,104.2,6108,476,339,130.7,856,201,543,117,14.4,1.7,4.1,0.8,22656.0,905.0,19.8,1.8,17.8,0.2,23.4,0.1,14.2,1.0,7.0,1.3,12.9,0.4,1.7,0.4,0.9,0.4,25.2,2.0,1.6,0.6,4.2,1.0,0.9,0.2,0.4723,0.2611,0.7561,0.8405,2.3300,0.6143,0.4715,0.6406,0.3763,0.2961,1.7845,0.3187,0.4206,0.7170,1.1376,0.5915,0.1512,0.8816,0.3703,0.2420,0.1165,1.7616,0.1741,7.0137,0.4242,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6303,732,11.0,1.3,40036
3,1,ALABAMA,AL,Butler,1013,"Butler County, Alabama",776.838201,20025,0,10026,51,6708,274,4640,521,567,147,20430,1258,2141,268,3806,21,4566,88,3492,370,704,143.9,9641,22.0,93,137.4,134,47.4,2625,212,119,57.7,520,102,322,88,23.5,2.6,6.7,1.7,20430.0,1258.0,15.4,1.9,19.0,0.1,22.8,0.4,17.7,1.9,10.5,2.1,48.1,0.1,0.5,0.7,1.3,0.5,26.2,2.1,1.8,0.9,7.8,1.5,1.6,0.4,0.8860,0.6968,0.8879,0.6753,3.1460,0.8455,0.5928,0.5578,0.6845,0.8185,2.6536,0.9077,0.8711,0.3582,1.2293,0.6447,0.2416,0.8940,0.4457,0.7685,0.3792,2.7290,0.6259,9.7579,0.8653,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2005,340,10.2,1.7,17280
4,1,ALABAMA,AL,Calhoun,1015,"Calhoun County, Alabama",605.867251,115098,0,53682,184,45033,683,20819,1317,4628,526,24706,758,12620,766,19386,119,25196,66,23598,1086,4701,464.0,31675,34.0,1076,250.2,1990,303.0,7904,546,772,206.2,2599,331,3112,436,18.6,1.2,8.8,1.0,24706.0,758.0,15.9,1.0,16.8,0.1,21.9,0.1,20.8,0.9,10.4,1.0,27.5,0.0,1.0,0.2,3.7,0.6,14.7,1.0,1.7,0.5,5.8,0.7,2.7,0.4,0.7322,0.8850,0.6076,0.6988,2.9236,0.7866,0.3664,0.4323,0.8

In [4]:
svi.to_csv('data_compressed/SVI2018_US_COUNTY.csv.gz', compression='gzip', index=False)

In [5]:
svi = pd.read_csv('data_compressed/SVI2018_US_COUNTY.csv.gz', compression='gzip')
svi.head()

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,71,12398,439,-999,-999,-999,-999,-999,-999,3669,426,7083,25,9318,14,6280,495,1330,285.0,34397,145.0,755,209.5,67,37.1,7770,431,264,77.1,763,160,654,142,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,13.8,1.6,18.0,0.1,23.7,0.0,16.1,1.3,10.7,2.3,87.5,0.4,2.1,0.6,0.3,0.2,38.8,2.2,2.1,0.6,6.2,1.3,1.7,0.4,-999.0000,-999.0000,-999.0000,0.5922,-999.0000,-999.0000,0.4893,0.6826,0.5610,0.8383,2.5712,0.8758,0.9917,0.7740,1.7657,0.9268,0.0551,0.9869,0.5498,0.5788,0.4126,2.5832,0.5409,-999.0000,-999.0000,-999,-999,-999,0,-999,0,0,0,0,0,1,0,1,0,1,0,0,0,1,-999,4160,588,10.6,1.5,32290
1,1,ALABAMA,AL,Autauga,1001,"Autauga County, Alabama",594.443459,55200,0,23315,71,21115,383,8422,1137,1065,257,29372,2306,4204,475,8050,75,13369,32,10465,729,1586,319.9,13788,59.0,426,205.9,886,308.7,4279,469,299,142.3,1191,272,546,161,15.4,2.1,4.2,1.0,29372.0,2306.0,11.3,1.3,14.6,0.1,24.2,0.1,19.3,1.3,7.5,1.5,25.0,0.1,0.8,0.4,3.8,1.3,18.4,2.0,1.4,0.7,5.6,1.3,1.0,0.3,0.5401,0.2745,0.2860,0.4397,1.5403,0.3631,0.1850,0.7529,0.7905,0.3792,2.1076,0.5810,0.6336,0.5113,1.1449,0.5947,0.6017,0.7408,0.2964,0.4846,0.1525,2.2760,0.3741,7.0688,0.4354,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3875,508,7.1,0.9,37301
2,1,ALABAMA,AL,Blount,1009,"Blount County, Alabama",644.830460,57645,0,24222,55,20600,396,8220,992,909,193,22656,905,7861,727,10233,91,13468,53,8114,592,1437,267.2,7413,229.0,934,239.3,211,104.2,6108,476,339,130.7,856,201,543,117,14.4,1.7,4.1,0.8,22656.0,905.0,19.8,1.8,17.8,0.2,23.4,0.1,14.2,1.0,7.0,1.3,12.9,0.4,1.7,0.4,0.9,0.4,25.2,2.0,1.6,0.6,4.2,1.0,0.9,0.2,0.4723,0.2611,0.7561,0.8405,2.3300,0.6143,0.4715,0.6406,0.3763,0.2961,1.7845,0.3187,0.4206,0.7170,1.1376,0.5915,0.1512,0.8816,0.3703,0.2420,0.1165,1.7616,0.1741,7.0137,0.4242,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6303,732,11.0,1.3,40036
3,1,ALABAMA,AL,Butler,1013,"Butler County, Alabama",776.838201,20025,0,10026,51,6708,274,4640,521,567,147,20430,1258,2141,268,3806,21,4566,88,3492,370,704,143.9,9641,22.0,93,137.4,134,47.4,2625,212,119,57.7,520,102,322,88,23.5,2.6,6.7,1.7,20430.0,1258.0,15.4,1.9,19.0,0.1,22.8,0.4,17.7,1.9,10.5,2.1,48.1,0.1,0.5,0.7,1.3,0.5,26.2,2.1,1.8,0.9,7.8,1.5,1.6,0.4,0.8860,0.6968,0.8879,0.6753,3.1460,0.8455,0.5928,0.5578,0.6845,0.8185,2.6536,0.9077,0.8711,0.3582,1.2293,0.6447,0.2416,0.8940,0.4457,0.7685,0.3792,2.7290,0.6259,9.7579,0.8653,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2005,340,10.2,1.7,17280
4,1,ALABAMA,AL,Calhoun,1015,"Calhoun County, Alabama",605.867251,115098,0,53682,184,45033,683,20819,1317,4628,526,24706,758,12620,766,19386,119,25196,66,23598,1086,4701,464.0,31675,34.0,1076,250.2,1990,303.0,7904,546,772,206.2,2599,331,3112,436,18.6,1.2,8.8,1.0,24706.0,758.0,15.9,1.0,16.8,0.1,21.9,0.1,20.8,0.9,10.4,1.0,27.5,0.0,1.0,0.2,3.7,0.6,14.7,1.0,1.7,0.5,5.8,0.7,2.7,0.4,0.7322,0.8850,0.6076,0.6988,2.9236,0.7866,0.3664,0.4323,0.8

**Vaccine Hesitancy:**

In [6]:
vax_hesitancy = pd.read_csv('data/Vaccine_Hesitancy_for_COVID-19__County_and_local_estimates.csv')
vax_hesitancy.head()

,FIPS Code,County Name,State,Estimated hesitant,Estimated hesitant or unsure,Estimated strongly hesitant,Social Vulnerability Index (SVI),SVI Category,CVAC level of concern for vaccination rollout,CVAC Level Of Concern,Percent adults fully vaccinated against COVID-19 (as of 6/10/21),Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Geographical Point,State Code,County Boundary,State Boundary
0,1123,"Tallapoosa County, Alabama",ALABAMA,0.1806,0.2400,0.1383,0.89,Very High Vulnerability,0.64,High Concern,0.305,0.0242,0.0022,0.0036,0.2697,0.0000,0.6887,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-85.841259 33.104456, -85.8409...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
1,1121,"Talladega County, Alabama",ALABAMA,0.1783,0.2350,0.1368,0.87,Very High Vulnerability,0.84,Very High Concern,0.265,0.0229,0.0043,0.0061,0.3237,0.0003,0.6263,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-86.303069 33.46316, -86.30306...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
2,1131,"Wilcox County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.93,Very High Vulnerability,0.94,Very High Concern,0.394,0.0053,0.0009,0.0003,0.6938,0.0000,0.2684,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.52534299999999 32.132773, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
3,1129,"Washington County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.73,High Vulnerability,0.82,Very High Concern,0.308,0.0146,0.0731,0.0025,0.2354,0.0000,0.6495,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-88.45317899999999 31.505388, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
4,1133,"Winston County, Alabama",ALABAMA,0.1805,0.2313,0.1379,0.70,High Vulnerability,0.80,High Concern,0.163,0.0315,0.0034,0.0016,0.0073,0.0005,0.9370,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.63656399999999 34.120908, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."


In [7]:
vax_hesitancy.to_csv('data_compressed/Vaccine_Hesitancy_for_COVID-19__County_and_local_estimates.csv.gz', 
                    compression='gzip', index=False)

In [8]:
vax_hesitancy = pd.read_csv('data_compressed/Vaccine_Hesitancy_for_COVID-19__County_and_local_estimates.csv.gz')
vax_hesitancy.head()

,FIPS Code,County Name,State,Estimated hesitant,Estimated hesitant or unsure,Estimated strongly hesitant,Social Vulnerability Index (SVI),SVI Category,CVAC level of concern for vaccination rollout,CVAC Level Of Concern,Percent adults fully vaccinated against COVID-19 (as of 6/10/21),Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Geographical Point,State Code,County Boundary,State Boundary
0,1123,"Tallapoosa County, Alabama",ALABAMA,0.1806,0.2400,0.1383,0.89,Very High Vulnerability,0.64,High Concern,0.305,0.0242,0.0022,0.0036,0.2697,0.0000,0.6887,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-85.841259 33.104456, -85.8409...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
1,1121,"Talladega County, Alabama",ALABAMA,0.1783,0.2350,0.1368,0.87,Very High Vulnerability,0.84,Very High Concern,0.265,0.0229,0.0043,0.0061,0.3237,0.0003,0.6263,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-86.303069 33.46316, -86.30306...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
2,1131,"Wilcox County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.93,Very High Vulnerability,0.94,Very High Concern,0.394,0.0053,0.0009,0.0003,0.6938,0.0000,0.2684,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.52534299999999 32.132773, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
3,1129,"Washington County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.73,High Vulnerability,0.82,Very High Concern,0.308,0.0146,0.0731,0.0025,0.2354,0.0000,0.6495,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-88.45317899999999 31.505388, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
4,1133,"Winston County, Alabama",ALABAMA,0.1805,0.2313,0.1379,0.70,High Vulnerability,0.80,High Concern,0.163,0.0315,0.0034,0.0016,0.0073,0.0005,0.9370,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.63656399999999 34.120908, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."


**County Health Rankings:**

In [9]:
county_health_rankings = pd.read_csv('data/countyhealthrankings-analytic_data2021.csv', header=[0,1])
county_health_rankings.head()

,State FIPS Code,County FIPS Code,5-digit FIPS Code,State Abbreviation,Name,Release Year,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,Premature death CI low,Premature death CI high,Premature death flag (0 = No Flag/1=Unreliable/2=Suppressed),Premature death (AIAN),Premature death CI low (AIAN),Premature death CI high (AIAN),Premature death flag (AIAN) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Asian/Pacific Islander),Premature death CI low (Asian/Pacific Islander),Premature death CI high (Asian/Pacific Islander),Premature death flag (Asian/Pacific Islander) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Black),Premature death CI low (Black),Premature death CI high (Black),Premature death flag (Black) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Hispanic),Premature death CI low (Hispanic),Premature death CI high (Hispanic),Premature death flag (Hispanic) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (White),Premature death CI low (White),Premature death CI high (White),Premature death flag (White) (. = No Flag/1=Unreliable/2=Suppressed),Poor or fair health raw value,Poor or fair health numerator,Poor or fair health denominator,Poor or fair health CI low,Poor or fair health CI high,Poor physical health days raw value,Poor physical health days numerator,Poor physical health days denominator,Poor physical health days CI low,Poor physical health days CI high,Poor mental health days raw value,Poor mental health days numerator,Poor mental health days denominator,Poor mental health days CI low,Poor mental health days CI high,Low birthweight raw value,Low birthweight numerator,Low birthweight denominator,Low birthweight CI low,Low birthweight CI high,LBW unreliable indicator (Unreliable = Numerator < 20 or relative standard error > 20%),Low birthweight (AIAN),Low birthweight CI low (AIAN),Low birthweight CI high (AIAN),Low birthweight (Asian/Pacific Islander),Low birthweight CI low (Asian/Pacific Islander),Low birthweight CI high (Asian/Pacific Islander),Low birthweight (Black),Low birthweight CI low (Black),Low birthweight CI high (Black),Low birthweight (Hispanic),Low birthweight CI low (Hispanic),Low birthweight CI high (Hispanic),Low birthweight (White),Low birthweight CI low (White),Low birthweight CI high (White),Adult smoking raw value,Adult smoking numerator,Adult smoking denominator,Adult smoking CI low,Adult smoking CI high,Adult obesity raw value,Adult obesity numerator,Adult obesity denominator,Adult obesity CI low,Adult obesity CI high,Food environment index raw value,Food environment index numerator,Food environment index denominator,Food environment index CI low,Food environment index CI high,Physical inactivity raw value,Physical inactivity numerator,Physical inactivity denominator,Physical inactivity CI low,Physical inactivity CI high,Access to exercise opportunities raw value,Access to exercise opportunities numerator,Access to exercise opportunities denominator,Access to exercise opportunities CI low,Access to exercise opportunities CI high,Excessive drinking raw value,Excessive drinking numerator,Excessive drinking denominator,Excessive drinking CI low,Excessive drinking CI high,Alcohol-impaired driving deaths raw value,Alcohol-impaired driving deaths numerator,Alcohol-impaired driving deaths denominator,Alcohol-impaired driving deaths CI low,Alcohol-impaired driving deaths CI high,Sexually transmitted infections raw value,Sexually transmitted infections numerator,Sexually transmitted infections denominator,Sexually transmitted infections CI low,Sexually transmitted infections CI high,Teen births raw value,Teen births numerator,Teen births denominator,Teen births CI low,Teen births CI high,Teen births (AIAN),Teen births CI low (AIAN),Teen births CI high (AIAN),Teen births (Asian/Pacific Islander),Teen births CI low (Asian/Pacific Islander),Teen births CI high (Asian/Pacific Islander),Teen births (Black),Teen b

In [10]:
county_health_rankings.to_csv('data_compressed/countyhealthrankings-analytic_data2021.csv.gz', compression='gzip',
                             index=False)

In [11]:
county_health_rankings = pd.read_csv('data_compressed/countyhealthrankings-analytic_data2021.csv.gz', 
                                     compression='gzip', header=[0,1])
county_health_rankings.head()

,State FIPS Code,County FIPS Code,5-digit FIPS Code,State Abbreviation,Name,Release Year,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,Premature death CI low,Premature death CI high,Premature death flag (0 = No Flag/1=Unreliable/2=Suppressed),Premature death (AIAN),Premature death CI low (AIAN),Premature death CI high (AIAN),Premature death flag (AIAN) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Asian/Pacific Islander),Premature death CI low (Asian/Pacific Islander),Premature death CI high (Asian/Pacific Islander),Premature death flag (Asian/Pacific Islander) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Black),Premature death CI low (Black),Premature death CI high (Black),Premature death flag (Black) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Hispanic),Premature death CI low (Hispanic),Premature death CI high (Hispanic),Premature death flag (Hispanic) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (White),Premature death CI low (White),Premature death CI high (White),Premature death flag (White) (. = No Flag/1=Unreliable/2=Suppressed),Poor or fair health raw value,Poor or fair health numerator,Poor or fair health denominator,Poor or fair health CI low,Poor or fair health CI high,Poor physical health days raw value,Poor physical health days numerator,Poor physical health days denominator,Poor physical health days CI low,Poor physical health days CI high,Poor mental health days raw value,Poor mental health days numerator,Poor mental health days denominator,Poor mental health days CI low,Poor mental health days CI high,Low birthweight raw value,Low birthweight numerator,Low birthweight denominator,Low birthweight CI low,Low birthweight CI high,LBW unreliable indicator (Unreliable = Numerator < 20 or relative standard error > 20%),Low birthweight (AIAN),Low birthweight CI low (AIAN),Low birthweight CI high (AIAN),Low birthweight (Asian/Pacific Islander),Low birthweight CI low (Asian/Pacific Islander),Low birthweight CI high (Asian/Pacific Islander),Low birthweight (Black),Low birthweight CI low (Black),Low birthweight CI high (Black),Low birthweight (Hispanic),Low birthweight CI low (Hispanic),Low birthweight CI high (Hispanic),Low birthweight (White),Low birthweight CI low (White),Low birthweight CI high (White),Adult smoking raw value,Adult smoking numerator,Adult smoking denominator,Adult smoking CI low,Adult smoking CI high,Adult obesity raw value,Adult obesity numerator,Adult obesity denominator,Adult obesity CI low,Adult obesity CI high,Food environment index raw value,Food environment index numerator,Food environment index denominator,Food environment index CI low,Food environment index CI high,Physical inactivity raw value,Physical inactivity numerator,Physical inactivity denominator,Physical inactivity CI low,Physical inactivity CI high,Access to exercise opportunities raw value,Access to exercise opportunities numerator,Access to exercise opportunities denominator,Access to exercise opportunities CI low,Access to exercise opportunities CI high,Excessive drinking raw value,Excessive drinking numerator,Excessive drinking denominator,Excessive drinking CI low,Excessive drinking CI high,Alcohol-impaired driving deaths raw value,Alcohol-impaired driving deaths numerator,Alcohol-impaired driving deaths denominator,Alcohol-impaired driving deaths CI low,Alcohol-impaired driving deaths CI high,Sexually transmitted infections raw value,Sexually transmitted infections numerator,Sexually transmitted infections denominator,Sexually transmitted infections CI low,Sexually transmitted infections CI high,Teen births raw value,Teen births numerator,Teen births denominator,Teen births CI low,Teen births CI high,Teen births (AIAN),Teen births CI low (AIAN),Teen births CI high (AIAN),Teen births (Asian/Pacific Islander),Teen births CI low (Asian/Pacific Islander),Teen births CI high (Asian/Pacific Islander),Teen births (Black),Teen b

**NYT COVID-19 Case & Death Counts:**

In [12]:
nyt_cases_deaths = pd.read_csv('data/NYT-covid-cases-deaths.csv')
nyt_cases_deaths

,date,county,state,fips,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths
0,2021-07-07,Autauga,Alabama,1001.0,7262,113.0,6082.0,101.0,1180.0,12.0
1,2021-07-07,Baldwin,Alabama,1003.0,22043,315.0,15543.0,238.0,6500.0,77.0
2,2021-07-07,Barbour,Alabama,1005.0,2347,60.0,1316.0,42.0,1031.0,18.0
3,2021-07-07,Bibb,Alabama,1007.0,2693,64.0,2158.0,40.0,535.0,24.0
4,2021-07-07,Blount,Alabama,1009.0,6988,139.0,5357.0,118.0,1631.0,21.0
...,...,...,...,...,...,...,...,...,...,...
3241,2021-07-07,Sweetwater,Wyoming,56037.0,4801,40.0,4553.0,NaN,248.0,NaN
3242,2021-07-07,Teton,Wyoming,56039.0,3804,11.0,3715.0,NaN,89.0,NaN
3243,2021-07-07,Uinta,Wyoming,56041.0,2325,13.0,1995.0,NaN,330.0,NaN
3244,2021-07-07,Washakie,Wyoming,56043.0,926,26.0,725.0,NaN,201.0,NaN


In [13]:
nyt_cases_deaths.to_csv('data_compressed/NYT-covid-cases-deaths.csv.gz', index=False, compression='gzip')

In [14]:
nyt_cases_deaths = pd.read_csv('data_compressed/NYT-covid-cases-deaths.csv.gz', compression='gzip')
nyt_cases_deaths

,date,county,state,fips,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths
0,2021-07-07,Autauga,Alabama,1001.0,7262,113.0,6082.0,101.0,1180.0,12.0
1,2021-07-07,Baldwin,Alabama,1003.0,22043,315.0,15543.0,238.0,6500.0,77.0
2,2021-07-07,Barbour,Alabama,1005.0,2347,60.0,1316.0,42.0,1031.0,18.0
3,2021-07-07,Bibb,Alabama,1007.0,2693,64.0,2158.0,40.0,535.0,24.0
4,2021-07-07,Blount,Alabama,1009.0,6988,139.0,5357.0,118.0,1631.0,21.0
...,...,...,...,...,...,...,...,...,...,...
3241,2021-07-07,Sweetwater,Wyoming,56037.0,4801,40.0,4553.0,NaN,248.0,NaN
3242,2021-07-07,Teton,Wyoming,56039.0,3804,11.0,3715.0,NaN,89.0,NaN
3243,2021-07-07,Uinta,Wyoming,56041.0,2325,13.0,1995.0,NaN,330.0,NaN
3244,2021-07-07,Washakie,Wyoming,56043.0,926,26.0,725.0,NaN,201.0,NaN


## Explore Data Sources

Each file will now be explored and basic preprocessing steps like indexing, relabeling, setting data types, and dropping unnecessary features will be completed.

___ 


**SOCIAL VULNERABILITY INDEX**

Source: [CDC SVI 2018 Documentation](https://www.atsdr.cdc.gov/placeandhealth/svi/documentation/SVI_documentation_2018.html)

The degree to which a community exhibits certain social conditions, including high poverty, low percentage of vehicle access, or crowded households, may affect that community’s ability to prevent human suffering and financial loss in the event of disaster. These factors describe a community’s *social vulnerability.*

The Social Vulnerability Index (SVI) was created by the CDC to help public health officials and emergency response planners identify and map the communities that will most likely need support before, during, and after a hazardous event. While usually ranked at the US Census tract level, SVI 2018, used in this analysis, ranks at the county level. Each county received a ranking for each of the 15 social factors, for each of the four themes, as well as one overall ranking. 

Variables used:
<img src='images/SVI_variables.jpg'>

<center> Social Vulnerability Index Measures <br> <i> Source: <a href="https://www.atsdr.cdc.gov/placeandhealth/svi/documentation/SVI_documentation_2018.html">CDC ATSDR</a></i></center>

In [15]:
svi.head()

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,71,12398,439,-999,-999,-999,-999,-999,-999,3669,426,7083,25,9318,14,6280,495,1330,285.0,34397,145.0,755,209.5,67,37.1,7770,431,264,77.1,763,160,654,142,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,13.8,1.6,18.0,0.1,23.7,0.0,16.1,1.3,10.7,2.3,87.5,0.4,2.1,0.6,0.3,0.2,38.8,2.2,2.1,0.6,6.2,1.3,1.7,0.4,-999.0000,-999.0000,-999.0000,0.5922,-999.0000,-999.0000,0.4893,0.6826,0.5610,0.8383,2.5712,0.8758,0.9917,0.7740,1.7657,0.9268,0.0551,0.9869,0.5498,0.5788,0.4126,2.5832,0.5409,-999.0000,-999.0000,-999,-999,-999,0,-999,0,0,0,0,0,1,0,1,0,1,0,0,0,1,-999,4160,588,10.6,1.5,32290
1,1,ALABAMA,AL,Autauga,1001,"Autauga County, Alabama",594.443459,55200,0,23315,71,21115,383,8422,1137,1065,257,29372,2306,4204,475,8050,75,13369,32,10465,729,1586,319.9,13788,59.0,426,205.9,886,308.7,4279,469,299,142.3,1191,272,546,161,15.4,2.1,4.2,1.0,29372.0,2306.0,11.3,1.3,14.6,0.1,24.2,0.1,19.3,1.3,7.5,1.5,25.0,0.1,0.8,0.4,3.8,1.3,18.4,2.0,1.4,0.7,5.6,1.3,1.0,0.3,0.5401,0.2745,0.2860,0.4397,1.5403,0.3631,0.1850,0.7529,0.7905,0.3792,2.1076,0.5810,0.6336,0.5113,1.1449,0.5947,0.6017,0.7408,0.2964,0.4846,0.1525,2.2760,0.3741,7.0688,0.4354,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3875,508,7.1,0.9,37301
2,1,ALABAMA,AL,Blount,1009,"Blount County, Alabama",644.830460,57645,0,24222,55,20600,396,8220,992,909,193,22656,905,7861,727,10233,91,13468,53,8114,592,1437,267.2,7413,229.0,934,239.3,211,104.2,6108,476,339,130.7,856,201,543,117,14.4,1.7,4.1,0.8,22656.0,905.0,19.8,1.8,17.8,0.2,23.4,0.1,14.2,1.0,7.0,1.3,12.9,0.4,1.7,0.4,0.9,0.4,25.2,2.0,1.6,0.6,4.2,1.0,0.9,0.2,0.4723,0.2611,0.7561,0.8405,2.3300,0.6143,0.4715,0.6406,0.3763,0.2961,1.7845,0.3187,0.4206,0.7170,1.1376,0.5915,0.1512,0.8816,0.3703,0.2420,0.1165,1.7616,0.1741,7.0137,0.4242,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6303,732,11.0,1.3,40036
3,1,ALABAMA,AL,Butler,1013,"Butler County, Alabama",776.838201,20025,0,10026,51,6708,274,4640,521,567,147,20430,1258,2141,268,3806,21,4566,88,3492,370,704,143.9,9641,22.0,93,137.4,134,47.4,2625,212,119,57.7,520,102,322,88,23.5,2.6,6.7,1.7,20430.0,1258.0,15.4,1.9,19.0,0.1,22.8,0.4,17.7,1.9,10.5,2.1,48.1,0.1,0.5,0.7,1.3,0.5,26.2,2.1,1.8,0.9,7.8,1.5,1.6,0.4,0.8860,0.6968,0.8879,0.6753,3.1460,0.8455,0.5928,0.5578,0.6845,0.8185,2.6536,0.9077,0.8711,0.3582,1.2293,0.6447,0.2416,0.8940,0.4457,0.7685,0.3792,2.7290,0.6259,9.7579,0.8653,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2005,340,10.2,1.7,17280
4,1,ALABAMA,AL,Calhoun,1015,"Calhoun County, Alabama",605.867251,115098,0,53682,184,45033,683,20819,1317,4628,526,24706,758,12620,766,19386,119,25196,66,23598,1086,4701,464.0,31675,34.0,1076,250.2,1990,303.0,7904,546,772,206.2,2599,331,3112,436,18.6,1.2,8.8,1.0,24706.0,758.0,15.9,1.0,16.8,0.1,21.9,0.1,20.8,0.9,10.4,1.0,27.5,0.0,1.0,0.2,3.7,0.6,14.7,1.0,1.7,0.5,5.8,0.7,2.7,0.4,0.7322,0.8850,0.6076,0.6988,2.9236,0.7866,0.3664,0.4323,0.8

In [16]:
svi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Columns: 123 entries, ST to E_DAYPOP
dtypes: float64(63), int64(56), object(4)
memory usage: 2.9+ MB


In [17]:
# check for duplicate fips value
svi.FIPS.value_counts().sort_values()

47103    1
47071    1
47089    1
47101    1
47099    1
        ..
47111    1
47109    1
47107    1
47105    1
8115     1
Name: FIPS, Length: 3142, dtype: int64

In [18]:
# set fips code as index
svi.set_index('FIPS', inplace=True)

In [19]:
list(svi.columns)

['ST',
 'STATE',
 'ST_ABBR',
 'COUNTY',
 'LOCATION',
 'AREA_SQMI',
 'E_TOTPOP',
 'M_TOTPOP',
 'E_HU',
 'M_HU',
 'E_HH',
 'M_HH',
 'E_POV',
 'M_POV',
 'E_UNEMP',
 'M_UNEMP',
 'E_PCI',
 'M_PCI',
 'E_NOHSDP',
 'M_NOHSDP',
 'E_AGE65',
 'M_AGE65',
 'E_AGE17',
 'M_AGE17',
 'E_DISABL',
 'M_DISABL',
 'E_SNGPNT',
 'M_SNGPNT',
 'E_MINRTY',
 'M_MINRTY',
 'E_LIMENG',
 'M_LIMENG',
 'E_MUNIT',
 'M_MUNIT',
 'E_MOBILE',
 'M_MOBILE',
 'E_CROWD',
 'M_CROWD',
 'E_NOVEH',
 'M_NOVEH',
 'E_GROUPQ',
 'M_GROUPQ',
 'EP_POV',
 'MP_POV',
 'EP_UNEMP',
 'MP_UNEMP',
 'EP_PCI',
 'MP_PCI',
 'EP_NOHSDP',
 'MP_NOHSDP',
 'EP_AGE65',
 'MP_AGE65',
 'EP_AGE17',
 'MP_AGE17',
 'EP_DISABL',
 'MP_DISABL',
 'EP_SNGPNT',
 'MP_SNGPNT',
 'EP_MINRTY',
 'MP_MINRTY',
 'EP_LIMENG',
 'MP_LIMENG',
 'EP_MUNIT',
 'MP_MUNIT',
 'EP_MOBILE',
 'MP_MOBILE',
 'EP_CROWD',
 'MP_CROWD',
 'EP_NOVEH',
 'MP_NOVEH',
 'EP_GROUPQ',
 'MP_GROUPQ',
 'EPL_POV',
 'EPL_UNEMP',
 'EPL_PCI',
 'EPL_NOHSDP',
 'SPL_THEME1',
 'RPL_THEME1',
 'EPL_AGE65',
 'EPL_AGE17'

For this analysis, I decided to drop the following features:
- margin of error
- raw values before controlling for population
- percentile rankings for each variable (instead, opted to use the percentages)

In [20]:
# drop margin of error columns for this analysis
svi_drops = ['ST', 'LOCATION', 'M_TOTPOP',  'M_HU', 'M_HH', 'M_POV', 'M_UNEMP', 'M_PCI', 'M_NOHSDP',
             'M_AGE65', 'M_AGE17', 'M_DISABL', 'M_SNGPNT', 'M_MINRTY', 'M_LIMENG', 'M_MUNIT',
             'M_MOBILE', 'M_CROWD', 'M_NOVEH', 'M_GROUPQ', 'MP_POV', 'MP_UNEMP', 'MP_PCI',
             'MP_NOHSDP', 'MP_AGE65', 'MP_AGE17', 'MP_DISABL', 'MP_SNGPNT', 'MP_MINRTY', 'MP_LIMENG',
             'MP_MUNIT', 'MP_MOBILE', 'MP_CROWD', 'MP_NOVEH', 'MP_GROUPQ',
             'EP_PCI', #EP_PCI is equal to E_PCI
             'E_POV', 'E_UNEMP', 'E_NOHSDP', 'E_AGE65', #dropping values that aren't population controlled
             'E_AGE17', 'E_DISABL', 'E_SNGPNT', 'E_MINRTY', 'E_LIMENG', 'E_MUNIT', 'E_MOBILE',
             'E_CROWD', 'E_NOVEH', 'E_GROUPQ',
             'E_UNINSUR', 'M_UNINSUR', 'MP_UNINSUR', 'E_DAYPOP',
             'EPL_POV', 'EPL_UNEMP', 'EPL_PCI', 'EPL_NOHSDP', 'SPL_THEME1', 'EPL_AGE65', 'EPL_AGE17',
             'EPL_DISABL', 'EPL_SNGPNT', 'SPL_THEME2', 'EPL_MINRTY', 'EPL_LIMENG', 'SPL_THEME3',
             'EPL_MUNIT', 'EPL_MOBILE', 'EPL_CROWD', 'EPL_NOVEH', 'EPL_GROUPQ', 'SPL_THEME4',
             'SPL_THEMES', 'F_POV', 'F_UNEMP', 'F_PCI', 'F_NOHSDP', 'F_THEME1', 'F_AGE65',
             'F_AGE17', 'F_DISABL', 'F_SNGPNT', 'F_THEME2', 'F_MINRTY', 'F_LIMENG', 'F_THEME3',
             'F_MUNIT', 'F_MOBILE', 'F_CROWD', 'F_NOVEH', 'F_GROUPQ', 'F_THEME4', 'F_TOTAL']
svi_mapper = {'E_TOTPOP': 'population_est',
              'E_HU': 'housing_units_est',
              'E_HH': 'households_est',
              'E_PCI': 'per_capita_income_est',
              'EP_POV': 'poverty_rate',
              'EP_UNEMP': 'unemployment_rate',
              'EP_NOHSDP': 'percent_adults_without_hs_diploma', #out of adults 25+
              'EP_AGE65': 'percent_pop_over_65',
              'EP_AGE17': 'percent_pop_under_17',
              'EP_DISABL': 'percent_pop_disability',
              'EP_SNGPNT': 'percent_households_single_parent', #with children under 18
              'EP_MINRTY': 'percent_pop_minority',
              'EP_LIMENG': 'percent_pop_limited_english', #persons 5+ who speak english "less than well"
              'EP_MUNIT': 'percentage_housing_multiunit', #percentage of housing in structures with 10+ units
              'EP_MOBILE': 'percentage_housing_mobile', #percentage of housing in mobile homes
              'EP_CROWD': 'percentage_housing_crowded', #percentage of households with more people than rooms
              'EP_NOVEH': 'percentage_households_without_vehicle',
              'EP_GROUPQ': 'percent_pop_in_group_quarters',
              'EP_UNINSUR': 'percent_pop_uninsured',
              'RPL_THEME1': 'socioeconomic_percentile_ranking',
              'RPL_THEME2': 'household_composition_and_disability_percentile_ranking',
              'RPL_THEME3': 'minority_status_and_language_percentile_ranking',
              'RPL_THEME4': 'housing_type_and_transportation_percentile_ranking',
              'RPL_THEMES': 'overall_percentil_ranking'}

In [21]:
svi.drop(columns=svi_drops, inplace=True)

In [22]:
svi.rename(axis=1, mapper=svi_mapper, inplace=True)

In [23]:
svi

,STATE,ST_ABBR,COUNTY,AREA_SQMI,population_est,housing_units_est,households_est,per_capita_income_est,poverty_rate,unemployment_rate,percent_adults_without_hs_diploma,percent_pop_over_65,percent_pop_under_17,percent_pop_disability,percent_households_single_parent,percent_pop_minority,percent_pop_limited_english,percentage_housing_multiunit,percentage_housing_mobile,percentage_housing_crowded,percentage_households_without_vehicle,percent_pop_in_group_quarters,socioeconomic_percentile_ranking,household_composition_and_disability_percentile_ranking,minority_status_and_language_percentile_ranking,housing_type_and_transportation_percentile_ranking,overall_percentil_ranking,percent_pop_uninsured
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,
35039,NEW MEXICO,NM,Rio Arriba,5860.869195,39307,20044,12398,-999,-999.0,-999.0,13.8,18.0,23.7,16.1,10.7,87.5,2.1,0.3,38.8,2.1,6.2,1.7,-999.0000,0.8758,0.9268,0.5409,-999.0000,10.6
1001,ALABAMA,AL,Autauga,594.443459,55200,23315,21115,29372,15.4,4.2,11.3,14.6,24.2,19.3,7.5,25.0,0.8,3.8,18.4,1.4,5.6,1.0,0.3631,0.5810,0.5947,0.3741,0.4354,7.1
1009,ALABAMA,AL,Blount,644.830460,57645,24222,20600,22656,14.4,4.1,19.8,17.8,23.4,14.2,7.0,12.9,1.7,0.9,25.2,1.6,4.2,0.9,0.6143,0.3187,0.5915,0.1741,0.4242,11.0
1013,ALABAMA,AL,Butler,776.838201,20025,10026,6708,20430,23.5,6.7,15.4,19.0,22.8,17.7,10.5,48.1,0.5,1.3,26.2,1.8,7.8,1.6,0.8455,0.9077,0.6447,0.6259,0.8653,10.2
1015,ALABAMA,AL,Calhoun,605.867251,115098,53682,45033,24706,18.6,8.8,15.9,16.8,21.9,20.8,10.4,27.5,1.0,3.7,14.7,1.7,5.8,2.7,0.7866,0.8303,0.6507,0.6492,0.8252,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48131,TEXAS,TX,Duval,1793.476183,11355,5592,3511,17864,25.6,10.6,32.9,17.8,26.1,26.2,15.8,90.9,5.7,0.5,13.0,7.4,10.0,4.4,0.9825,0.9990,0.9809,0.8978,0.9994,24.5
48505,TEXAS,TX,Zapata,998.411980,14369,6388,4405,17228,39.5,11.0,40.0,13.9,33.6,17.3,17.1,95.7,24.4,0.6,34.2,18.1,7.9,0.2,0.9962,0.9452,0.9990,0.6966,0.9895,29.3
48507,TEXAS,TX,Zavala,1297.406535,12131,4344,3509,13350,34.8,8.4,38.0,13.7,29.5,23.3,16.1,95.0,8.0,1.7,22.8,5.6,14.4,1.6,0.9847,0.9857,0.9917,0.9331,0.9987,18.4


Engineer a feature called "population_density" that represents the number of people per square mile

In [24]:
svi['population_density'] = svi['population_est'] / svi['AREA_SQMI']

In [25]:
svi

,STATE,ST_ABBR,COUNTY,AREA_SQMI,population_est,housing_units_est,households_est,per_capita_income_est,poverty_rate,unemployment_rate,percent_adults_without_hs_diploma,percent_pop_over_65,percent_pop_under_17,percent_pop_disability,percent_households_single_parent,percent_pop_minority,percent_pop_limited_english,percentage_housing_multiunit,percentage_housing_mobile,percentage_housing_crowded,percentage_households_without_vehicle,percent_pop_in_group_quarters,socioeconomic_percentile_ranking,household_composition_and_disability_percentile_ranking,minority_status_and_language_percentile_ranking,housing_type_and_transportation_percentile_ranking,overall_percentil_ranking,percent_pop_uninsured,population_density
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
35039,NEW MEXICO,NM,Rio Arriba,5860.869195,39307,20044,12398,-999,-999.0,-999.0,13.8,18.0,23.7,16.1,10.7,87.5,2.1,0.3,38.8,2.1,6.2,1.7,-999.0000,0.8758,0.9268,0.5409,-999.0000,10.6,6.706684
1001,ALABAMA,AL,Autauga,594.443459,55200,23315,21115,29372,15.4,4.2,11.3,14.6,24.2,19.3,7.5,25.0,0.8,3.8,18.4,1.4,5.6,1.0,0.3631,0.5810,0.5947,0.3741,0.4354,7.1,92.859967
1009,ALABAMA,AL,Blount,644.830460,57645,24222,20600,22656,14.4,4.1,19.8,17.8,23.4,14.2,7.0,12.9,1.7,0.9,25.2,1.6,4.2,0.9,0.6143,0.3187,0.5915,0.1741,0.4242,11.0,89.395591
1013,ALABAMA,AL,Butler,776.838201,20025,10026,6708,20430,23.5,6.7,15.4,19.0,22.8,17.7,10.5,48.1,0.5,1.3,26.2,1.8,7.8,1.6,0.8455,0.9077,0.6447,0.6259,0.8653,10.2,25.777569
1015,ALABAMA,AL,Calhoun,605.867251,115098,53682,45033,24706,18.6,8.8,15.9,16.8,21.9,20.8,10.4,27.5,1.0,3.7,14.7,1.7,5.8,2.7,0.7866,0.8303,0.6507,0.6492,0.8252,9.4,189.972308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48131,TEXAS,TX,Duval,1793.476183,11355,5592,3511,17864,25.6,10.6,32.9,17.8,26.1,26.2,15.8,90.9,5.7,0.5,13.0,7.4,10.0,4.4,0.9825,0.9990,0.9809,0.8978,0.9994,24.5,6.331280
48505,TEXAS,TX,Zapata,998.411980,14369,6388,4405,17228,39.5,11.0,40.0,13.9,33.6,17.3,17.1,95.7,24.4,0.6,34.2,18.1,7.9,0.2,0.9962,0.9452,0.9990,0.6966,0.9895,29.3,14.391855
48507,TEXAS,TX,Zavala,1297.406535,12131,4344,3509,13350,34.8,8.4,38.0,13.7,29.5,23.3,16.1,95.0,8.0,1.7,22.8,5.6,14.4,1.6,0.9847,0.9857,0.9917,0.9331,0.9987,18.4,9.350192


____

**VACCINE HESITANCY**

*Source: [US Dept of Health & Human Services ASPE](https://aspe.hhs.gov/pdf-report/vaccine-hesitancy)*

To support state and local communication and outreach efforts, ASPE developed state, county, and sub-state level predictions of hesitancy rates using the U.S. Census Bureau’s [Household Pulse Survey (HPS)](https://www.census.gov/programs-surveys/household-pulse-survey.html). The HPS is nationally representative and includes information on U.S. residents’ intentions to receive the COVID-19 vaccine when available, as well as other sociodemographic and geographic (state, region and metropolitan statistical areas) information. 

In [26]:
vax_hesitancy.head()

,FIPS Code,County Name,State,Estimated hesitant,Estimated hesitant or unsure,Estimated strongly hesitant,Social Vulnerability Index (SVI),SVI Category,CVAC level of concern for vaccination rollout,CVAC Level Of Concern,Percent adults fully vaccinated against COVID-19 (as of 6/10/21),Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Geographical Point,State Code,County Boundary,State Boundary
0,1123,"Tallapoosa County, Alabama",ALABAMA,0.1806,0.2400,0.1383,0.89,Very High Vulnerability,0.64,High Concern,0.305,0.0242,0.0022,0.0036,0.2697,0.0000,0.6887,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-85.841259 33.104456, -85.8409...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
1,1121,"Talladega County, Alabama",ALABAMA,0.1783,0.2350,0.1368,0.87,Very High Vulnerability,0.84,Very High Concern,0.265,0.0229,0.0043,0.0061,0.3237,0.0003,0.6263,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-86.303069 33.46316, -86.30306...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
2,1131,"Wilcox County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.93,Very High Vulnerability,0.94,Very High Concern,0.394,0.0053,0.0009,0.0003,0.6938,0.0000,0.2684,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.52534299999999 32.132773, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
3,1129,"Washington County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.73,High Vulnerability,0.82,Very High Concern,0.308,0.0146,0.0731,0.0025,0.2354,0.0000,0.6495,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-88.45317899999999 31.505388, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
4,1133,"Winston County, Alabama",ALABAMA,0.1805,0.2313,0.1379,0.70,High Vulnerability,0.80,High Concern,0.163,0.0315,0.0034,0.0016,0.0073,0.0005,0.9370,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.63656399999999 34.120908, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."


In [27]:
vax_hesitancy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 21 columns):
 #   Column                                                            Non-Null Count  Dtype  
---  ------                                                            --------------  -----  
 0   FIPS Code                                                         3142 non-null   int64  
 1   County Name                                                       3142 non-null   object 
 2   State                                                             3142 non-null   object 
 3   Estimated hesitant                                                3142 non-null   float64
 4   Estimated hesitant or unsure                                      3142 non-null   float64
 5   Estimated strongly hesitant                                       3142 non-null   float64
 6   Social Vulnerability Index (SVI)                                  3141 non-null   float64
 7   SVI Category                     

In [28]:
vax_hesitancy.set_index('FIPS Code', inplace=True)

In [29]:
# drop SVI columns since I will merge with the full SVI 2018 data 
vax_hesitancy.drop(columns=['Social Vulnerability Index (SVI)', 'SVI Category'], inplace=True)

In [30]:
# explore nulls
vax_hesitancy[vax_hesitancy['Percent adults fully vaccinated against COVID-19 (as of 6/10/21)'].isna()]

,County Name,State,Estimated hesitant,Estimated hesitant or unsure,Estimated strongly hesitant,CVAC level of concern for vaccination rollout,CVAC Level Of Concern,Percent adults fully vaccinated against COVID-19 (as of 6/10/21),Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Geographical Point,State Code,County Boundary,State Boundary
FIPS Code,,,,,,,,,,,,,,,,,,
15001,"Hawaii County, Hawaii",HAWAII,0.0638,0.1031,0.0334,0.35,Low Concern,NaN,0.1273,0.0018,0.2184,0.0058,0.1108,0.3031,POINT (-157.854309 21.361013),HI,"MULTIPOLYGON (((-155.432862 20.145207, -155.43...","MULTIPOLYGON (((-155.432862 20.145207, -155.39..."
15003,"Honolulu County, Hawaii",HAWAII,0.0507,0.0848,0.0275,0.30,Low Concern,NaN,0.0985,0.0013,0.4173,0.0225,0.0893,0.1837,POINT (-157.854309 21.361013),HI,"MULTIPOLYGON (((-158.328652 21.546, -158.33505...","MULTIPOLYGON (((-155.432862 20.145207, -155.39..."
15005,"Kalawao County, Hawaii",HAWAII,0.0610,0.0979,0.0326,0.57,Moderate Concern,NaN,0.0303,0.0152,0.2121,0.0303,0.2727,0.2273,POINT (-157.854309 21.361013),HI,"MULTIPOLYGON (((-157.015058 21.261194, -157.00...","MULTIPOLYGON (((-155.432862 20.145207, -155.39..."
48001,"Anderson County, Texas",TEXAS,0.1398,0.2105,0.0948,0.95,Very High Concern,NaN,0.1779,0.0031,0.0056,0.2095,0.0002,0.5893,POINT (-99.317341 31.447238),TX,"MULTIPOLYGON (((-95.677533 32.051631, -95.4879...","MULTIPOLYGON (((-103.98017899999999 32.000125,..."
48019,"Bandera County, Texas",TEXAS,0.1146,0.1740,0.0756,0.77,High Concern,NaN,0.1879,0.0035,0.0028,0.0069,0.0000,0.7783,POINT (-99.317341 31.447238),TX,"MULTIPOLYGON (((-99.262695 29.625704, -99.2634...","MULTIPOLYGON (((-103.98017899999999 32.000125,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51670,"Hopewell city, Virginia",VIRGINIA,0.0682,0.1444,0.0469,0.56,Moderate Concern,NaN,0.0793,0.0014,0.0137,0.4221,0.0000,0.4646,POINT (-78.666382 37.510861),VA,"MULTIPOLYGON (((-77.337774 37.306483, -77.3377...","MULTIPOLYGON (((-77.328318 39.057677999999996,..."
51580,"Covington city, Virginia",VIRGINIA,0.0574,0.1126,0.0394,0.29,Low Concern,NaN,0.0238,0.0000,0.0116,0.1249,0.0020,0.8099,POINT (-78.666382 37.510861),VA,"MULTIPOLYGON (((-80.008994 37.762586999999996,...","MULTIPOLYGON (((-77.328318 39.057677999999996,..."
51530,"Buena Vista city, Virginia",VIRGINIA,0.0678,0.1359,0.0467,0.36,Low Concern,NaN,0.0180,0.0009,0.0190,0.0728,0.0025,0.8812,POINT (-78.666382 37.510861),VA,"MULTIPOLYGON (((-79.381509 37.742149, -79.3811...","MULTIPOLYGON (((-77.328318 39.057677999999996,..."


Upon closer look, I'll remove the following columns as well:
- "CVAC level of concern" - remove the categorical column in favor of the continuous column 
- "% of adults fully vaccinated against COVID-19" - null values and expected multicollinearity with vaccine hesitancy
- State & County Boundaries and Geographical Point - will instead use a geojson to plot a choropleth map
- County Name, State, and State Code - avoid duplicates upon merge

In [31]:
vax_hesitancy.drop(columns=['CVAC Level Of Concern', 
                            'Percent adults fully vaccinated against COVID-19 (as of 6/10/21)',
                            'State Boundary', 'County Boundary', 'Geographical Point',
                            'County Name', 'State', 'State Code'], inplace=True)

In [32]:
vax_hesitancy

,Estimated hesitant,Estimated hesitant or unsure,Estimated strongly hesitant,CVAC level of concern for vaccination rollout,Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White
FIPS Code,,,,,,,,,,
1123,0.1806,0.2400,0.1383,0.64,0.0242,0.0022,0.0036,0.2697,0.0000,0.6887
1121,0.1783,0.2350,0.1368,0.84,0.0229,0.0043,0.0061,0.3237,0.0003,0.6263
1131,0.1735,0.2357,0.1337,0.94,0.0053,0.0009,0.0003,0.6938,0.0000,0.2684
1129,0.1735,0.2357,0.1337,0.82,0.0146,0.0731,0.0025,0.2354,0.0000,0.6495
1133,0.1805,0.2313,0.1379,0.80,0.0315,0.0034,0.0016,0.0073,0.0005,0.9370
...,...,...,...,...,...,...,...,...,...,...
55079,0.1505,0.2144,0.0880,0.30,0.1500,0.0047,0.0428,0.2606,0.0002,0.5124
55121,0.1611,0.2126,0.0959,0.31,0.0840,0.0034,0.0043,0.0051,0.0000,0.8953
56001,0.1949,0.2673,0.1405,0.63,0.0953,0.0091,0.0327,0.0150,0.0003,0.8248


_____

**COUNTY HEALTH RANKINGS**

The County Health Rankings are based on a model of community health that emphasizes the many factors that influence how long and how well we live. The Rankings use more than 30 measures that help communities understand how healthy their residents are today (health outcomes) and what will impact their health in the future (health factors). 

The County Health Rankings & Roadmaps is a program of the [University of Wisconsin Population Health Institute](https://uwphi.pophealth.wisc.edu/) supported by the [Robert Wood Johnson Foundation](https://www.rwjf.org/).

In [33]:
county_health_rankings.head()

,State FIPS Code,County FIPS Code,5-digit FIPS Code,State Abbreviation,Name,Release Year,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,Premature death CI low,Premature death CI high,Premature death flag (0 = No Flag/1=Unreliable/2=Suppressed),Premature death (AIAN),Premature death CI low (AIAN),Premature death CI high (AIAN),Premature death flag (AIAN) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Asian/Pacific Islander),Premature death CI low (Asian/Pacific Islander),Premature death CI high (Asian/Pacific Islander),Premature death flag (Asian/Pacific Islander) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Black),Premature death CI low (Black),Premature death CI high (Black),Premature death flag (Black) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Hispanic),Premature death CI low (Hispanic),Premature death CI high (Hispanic),Premature death flag (Hispanic) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (White),Premature death CI low (White),Premature death CI high (White),Premature death flag (White) (. = No Flag/1=Unreliable/2=Suppressed),Poor or fair health raw value,Poor or fair health numerator,Poor or fair health denominator,Poor or fair health CI low,Poor or fair health CI high,Poor physical health days raw value,Poor physical health days numerator,Poor physical health days denominator,Poor physical health days CI low,Poor physical health days CI high,Poor mental health days raw value,Poor mental health days numerator,Poor mental health days denominator,Poor mental health days CI low,Poor mental health days CI high,Low birthweight raw value,Low birthweight numerator,Low birthweight denominator,Low birthweight CI low,Low birthweight CI high,LBW unreliable indicator (Unreliable = Numerator < 20 or relative standard error > 20%),Low birthweight (AIAN),Low birthweight CI low (AIAN),Low birthweight CI high (AIAN),Low birthweight (Asian/Pacific Islander),Low birthweight CI low (Asian/Pacific Islander),Low birthweight CI high (Asian/Pacific Islander),Low birthweight (Black),Low birthweight CI low (Black),Low birthweight CI high (Black),Low birthweight (Hispanic),Low birthweight CI low (Hispanic),Low birthweight CI high (Hispanic),Low birthweight (White),Low birthweight CI low (White),Low birthweight CI high (White),Adult smoking raw value,Adult smoking numerator,Adult smoking denominator,Adult smoking CI low,Adult smoking CI high,Adult obesity raw value,Adult obesity numerator,Adult obesity denominator,Adult obesity CI low,Adult obesity CI high,Food environment index raw value,Food environment index numerator,Food environment index denominator,Food environment index CI low,Food environment index CI high,Physical inactivity raw value,Physical inactivity numerator,Physical inactivity denominator,Physical inactivity CI low,Physical inactivity CI high,Access to exercise opportunities raw value,Access to exercise opportunities numerator,Access to exercise opportunities denominator,Access to exercise opportunities CI low,Access to exercise opportunities CI high,Excessive drinking raw value,Excessive drinking numerator,Excessive drinking denominator,Excessive drinking CI low,Excessive drinking CI high,Alcohol-impaired driving deaths raw value,Alcohol-impaired driving deaths numerator,Alcohol-impaired driving deaths denominator,Alcohol-impaired driving deaths CI low,Alcohol-impaired driving deaths CI high,Sexually transmitted infections raw value,Sexually transmitted infections numerator,Sexually transmitted infections denominator,Sexually transmitted infections CI low,Sexually transmitted infections CI high,Teen births raw value,Teen births numerator,Teen births denominator,Teen births CI low,Teen births CI high,Teen births (AIAN),Teen births CI low (AIAN),Teen births CI high (AIAN),Teen births (Asian/Pacific Islander),Teen births CI low (Asian/Pacific Islander),Teen births CI high (Asian/Pacific Islander),Teen births (Black),Teen b

In [34]:
list(county_health_rankings.columns)

[('State FIPS Code', 'statecode'),
 ('County FIPS Code', 'countycode'),
 ('5-digit FIPS Code', 'fipscode'),
 ('State Abbreviation', 'state'),
 ('Name', 'county'),
 ('Release Year', 'year'),
 ('County Ranked (Yes=1/No=0)', 'county_ranked'),
 ('Premature death raw value', 'v001_rawvalue'),
 ('Premature death numerator', 'v001_numerator'),
 ('Premature death denominator', 'v001_denominator'),
 ('Premature death CI low', 'v001_cilow'),
 ('Premature death CI high', 'v001_cihigh'),
 ('Premature death flag (0 = No Flag/1=Unreliable/2=Suppressed)', 'v001_flag'),
 ('Premature death (AIAN)', 'v001_race_aian'),
 ('Premature death CI low (AIAN)', 'v001_race_aian_cilow'),
 ('Premature death CI high (AIAN)', 'v001_race_aian_cihigh'),
 ('Premature death flag (AIAN) (. = No Flag/1=Unreliable/2=Suppressed)',
  'v001_race_aian_flag'),
 ('Premature death (Asian/Pacific Islander)', 'v001_race_asian'),
 ('Premature death CI low (Asian/Pacific Islander)', 'v001_race_asian_cilow'),
 ('Premature death CI high

In [35]:
chr_cols = []
for item in list(county_health_rankings.columns):
    chr_cols.append(item[0])
chr_cols

['State FIPS Code',
 'County FIPS Code',
 '5-digit FIPS Code',
 'State Abbreviation',
 'Name',
 'Release Year',
 'County Ranked (Yes=1/No=0)',
 'Premature death raw value',
 'Premature death numerator',
 'Premature death denominator',
 'Premature death CI low',
 'Premature death CI high',
 'Premature death flag (0 = No Flag/1=Unreliable/2=Suppressed)',
 'Premature death (AIAN)',
 'Premature death CI low (AIAN)',
 'Premature death CI high (AIAN)',
 'Premature death flag (AIAN) (. = No Flag/1=Unreliable/2=Suppressed)',
 'Premature death (Asian/Pacific Islander)',
 'Premature death CI low (Asian/Pacific Islander)',
 'Premature death CI high (Asian/Pacific Islander)',
 'Premature death flag (Asian/Pacific Islander) (. = No Flag/1=Unreliable/2=Suppressed)',
 'Premature death (Black)',
 'Premature death CI low (Black)',
 'Premature death CI high (Black)',
 'Premature death flag (Black) (. = No Flag/1=Unreliable/2=Suppressed)',
 'Premature death (Hispanic)',
 'Premature death CI low (Hispanic

In [36]:
chr_keepcols = ['5-digit FIPS Code', 'State Abbreviation', 'Name', 'County Ranked (Yes=1/No=0)', 
                #health & healthy behaviors features
                'Premature death raw value', 'Poor or fair health raw value', 
                'Poor physical health days raw value', 'Poor mental health days raw value',
                'Adult smoking raw value', 'Adult obesity raw value', 'Physical inactivity raw value', 
                'Excessive drinking raw value', 'Preventable hospital stays raw value', 
                'Flu vaccinations raw value', 'Life expectancy raw value',
                #access to healthcare
                'Uninsured raw value', 'Ratio of population to primary care physicians.',
                #food 
                'Food environment index raw value', 'Food insecurity raw value',
                #housing
                'Homeownership raw value', 'Broadband access raw value',
                #employment & income
                'Unemployment raw value', 'Income inequality raw value', 'Median household income raw value',
                #environment
                'Air pollution - particulate matter raw value', 'Drinking water violations raw value',  
                '% Rural raw value'
               ]

In [37]:
chr_df = county_health_rankings[chr_keepcols].set_axis(axis=1, labels=chr_keepcols)

In [38]:
chr_df.set_index('5-digit FIPS Code', inplace=True)

In [39]:
chr_df

,State Abbreviation,Name,County Ranked (Yes=1/No=0),Premature death raw value,Poor or fair health raw value,Poor physical health days raw value,Poor mental health days raw value,Adult smoking raw value,Adult obesity raw value,Physical inactivity raw value,Excessive drinking raw value,Preventable hospital stays raw value,Flu vaccinations raw value,Life expectancy raw value,Uninsured raw value,Ratio of population to primary care physicians.,Food environment index raw value,Food insecurity raw value,Homeownership raw value,Broadband access raw value,Unemployment raw value,Income inequality raw value,Median household income raw value,Air pollution - particulate matter raw value,Drinking water violations raw value,% Rural raw value
5-digit FIPS Code,,,,,,,,,,,,,,,,,,,,,,,,,,
0,US,United States,NaN,6906.641094,0.165096,3.741935,4.062792,0.166225,0.297,0.227,0.191746,4236.0,0.48,79.173088,0.104289,1319.042688,7.8,0.115,0.639921,0.826665,0.036694,4.913801,65712.0,7.2,NaN,0.192690
1000,AL,Alabama,NaN,9819.887431,0.213758,4.416965,4.885636,0.199898,0.361,0.293,0.148914,5466.0,0.43,75.548075,0.118847,1533.690304,5.5,0.170,0.687806,0.763991,0.030006,5.238165,51771.0,9.2,0.104478,0.409632
1001,AL,Autauga County,1.0,7830.053484,0.198392,4.501499,4.850214,0.198108,0.330,0.306,0.144739,6650.0,0.42,77.162581,0.100473,2138.500000,6.7,0.156,0.732906,0.805674,0.027281,5.090408,58233.0,10.4,0.000000,0.420022
1003,AL,Baldwin County,1.0,7680.477270,0.164607,3.647978,4.768294,0.185133,0.300,0.247,0.187370,3471.0,0.46,78.213405,0.131690,1424.980392,7.8,0.129,0.752453,0.818164,0.027258,4.385135,59871.0,7.2,0.000000,0.422791
1005,AL,Barbour County,1.0,11476.629416,0.298415,5.569267,5.590494,0.255308,0.412,0.280,0.124670,5314.0,0.39,74.054741,0.135040,3110.125000,5.5,0.219,0.608989,0.605243,0.037952,5.981993,35972.0,9.4,0.000000,0.677896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,WY,Sweetwater County,1.0,7775.070675,0.164432,3.594866,3.980964,0.185086,0.303,0.244,0.191880,2543.0,0.42,77.533795,0.114610,2532.411765,7.5,0.117,0.762224,0.840044,0.038921,4.086863,80639.0,5.0,1.000000,0.109163
56039,WY,Teton County,1.0,2979.684888,0.114117,2.935199,3.337136,0.128844,0.110,0.108,0.237739,3181.0,0.53,87.471434,0.133664,923.240000,8.3,0.095,0.604058,0.871272,0.027223,3.992274,98837.0,4.5,1.000000,0.464309
56041,WY,Uinta County,1.0,8081.051208,0.169134,4.025276,4.239036,0.207645,0.352,0.251,0.184073,2839.0,0.35,78.244365,0.118385,2029.900000,7.3,0.135,0.770567,0.895353,0.039070,4.167413,70756.0,6.2,0.000000,0.430959


In [40]:
chr_df = chr_df[chr_df['County Ranked (Yes=1/No=0)']==1.0]
chr_df.drop(columns='County Ranked (Yes=1/No=0)', inplace=True)
chr_df.head()

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,State Abbreviation,Name,Premature death raw value,Poor or fair health raw value,Poor physical health days raw value,Poor mental health days raw value,Adult smoking raw value,Adult obesity raw value,Physical inactivity raw value,Excessive drinking raw value,Preventable hospital stays raw value,Flu vaccinations raw value,Life expectancy raw value,Uninsured raw value,Ratio of population to primary care physicians.,Food environment index raw value,Food insecurity raw value,Homeownership raw value,Broadband access raw value,Unemployment raw value,Income inequality raw value,Median household income raw value,Air pollution - particulate matter raw value,Drinking water violations raw value,% Rural raw value
5-digit FIPS Code,,,,,,,,,,,,,,,,,,,,,,,,,
1001,AL,Autauga County,7830.053484,0.198392,4.501499,4.850214,0.198108,0.330,0.306,0.144739,6650.0,0.42,77.162581,0.100473,2138.500000,6.7,0.156,0.732906,0.805674,0.027281,5.090408,58233.0,10.4,0.0,0.420022
1003,AL,Baldwin County,7680.477270,0.164607,3.647978,4.768294,0.185133,0.300,0.247,0.187370,3471.0,0.46,78.213405,0.131690,1424.980392,7.8,0.129,0.752453,0.818164,0.027258,4.385135,59871.0,7.2,0.0,0.422791
1005,AL,Barbour County,11476.629416,0.298415,5.569267,5.590494,0.255308,0.412,0.280,0.124670,5314.0,0.39,74.054741,0.135040,3110.125000,5.5,0.219,0.608989,0.605243,0.037952,5.981993,35972.0,9.4,0.0,0.677896
1007,AL,Bibb County,12172.562382,0.238533,4.894377,5.271114,0.230848,0.374,0.334,0.154206,6690.0,0.40,73.408784,0.105991,1866.666667,7.6,0.151,0.744159,0.692062,0.030628,5.002608,47918.0,10.0,0.0,0.683526
1009,AL,Blount County,11096.050649,0.219856,4.986622,5.357809,0.229550,0.330,0.333,0.158333,4440.0,0.40,74.370874,0.141160,4820.000000,7.9,0.136,0.787787,0.730081,0.026687,4.427838,52902.0,10.6,0.0,0.899515


____

**NEW YORK TIMES COVID-19 CASE & DEATH COUNTS**

The New York Times is releasing a series of data files with cumulative counts of coronavirus cases in the United States, at the state and county level, over time. Time series data is compiled by the New York Times from state and local governments and health departments in an attempt to provide a complete record of the ongoing outbreak.

For this analysis, we use the cumulative counts of probable and comfirmed cases and probable and confirmed deaths from COVID-19 as of July 7, 2021. In most cases, patients were attributed to the county they were treated, not necessarily where they live.

Data is posted and regularly updated in the [nytimes/covid-19-data](https://github.com/nytimes/covid-19-data)  github repo. The README contains additional definitions and details about the data collection methodology.

For answers to common questions, see their [FAQ.](https://www.nytimes.com/interactive/2020/us/about-coronavirus-data-maps.html)

In [41]:
nyt_cases_deaths

,date,county,state,fips,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths
0,2021-07-07,Autauga,Alabama,1001.0,7262,113.0,6082.0,101.0,1180.0,12.0
1,2021-07-07,Baldwin,Alabama,1003.0,22043,315.0,15543.0,238.0,6500.0,77.0
2,2021-07-07,Barbour,Alabama,1005.0,2347,60.0,1316.0,42.0,1031.0,18.0
3,2021-07-07,Bibb,Alabama,1007.0,2693,64.0,2158.0,40.0,535.0,24.0
4,2021-07-07,Blount,Alabama,1009.0,6988,139.0,5357.0,118.0,1631.0,21.0
...,...,...,...,...,...,...,...,...,...,...
3241,2021-07-07,Sweetwater,Wyoming,56037.0,4801,40.0,4553.0,NaN,248.0,NaN
3242,2021-07-07,Teton,Wyoming,56039.0,3804,11.0,3715.0,NaN,89.0,NaN
3243,2021-07-07,Uinta,Wyoming,56041.0,2325,13.0,1995.0,NaN,330.0,NaN
3244,2021-07-07,Washakie,Wyoming,56043.0,926,26.0,725.0,NaN,201.0,NaN


In [44]:
nyt_cases_deaths = nyt_cases_deaths[['county', 'state', 'fips', 'cases', 'deaths']]
nyt_cases_deaths.head()

,county,state,fips,cases,deaths
0,Autauga,Alabama,1001.0,7262,113.0
1,Baldwin,Alabama,1003.0,22043,315.0
2,Barbour,Alabama,1005.0,2347,60.0
3,Bibb,Alabama,1007.0,2693,64.0
4,Blount,Alabama,1009.0,6988,139.0


In [48]:
nyt_cases_deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3246 entries, 0 to 3245
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   county  3246 non-null   object 
 1   state   3246 non-null   object 
 2   fips    3216 non-null   float64
 3   cases   3246 non-null   int64  
 4   deaths  3168 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 126.9+ KB


There are some noted geographic exceptions that need to be reconciled before merging dataframes on fips code. Some are consequential to the analysis while others will be dropped.

In [51]:
#review rows missing fips code
nyt_cases_deaths[nyt_cases_deaths['fips'].isna()]

,county,state,fips,cases,deaths
71,Bristol Bay plus Lake and Peninsula,Alaska,NaN,201,0.0
90,Unknown,Alaska,NaN,299,0.0
93,Yakutat plus Hoonah-Angoon,Alaska,NaN,81,2.0
180,Unknown,Arkansas,NaN,2967,0.0
315,Unknown,Connecticut,NaN,1179,1.0
320,Unknown,Delaware,NaN,227,0.0
385,Unknown,Florida,NaN,56,1012.0
534,Unknown,Georgia,NaN,3041,4.0
550,Unknown,Guam,NaN,9363,141.0
691,Unknown,Illinois,NaN,0,432.0


Starting with the quite consequential New York City, I will use data from the New York State Department of Health to split cases attributed to NYC into the 4 burroughs: New York, Bronx, Kings, Queens, and Richmond counties. [Data source](https://covid19tracker.health.ny.gov/views/NYS-COVID19-Tracker/NYSDOHCOVID-19Tracker-TableView?%3Aembed=yes&%3Atoolbar=no&%3Atabs=n)

In [72]:
#cross-check case counts from NYT source and NY Dept of Health
nyt_cases_deaths[nyt_cases_deaths['state'] == 'New York']

,county,state,fips,cases,deaths
1845,Albany,New York,36001.0,24740,357.0
1846,Allegany,New York,36003.0,3562,87.0
1847,Broome,New York,36007.0,18669,366.0
1848,Cattaraugus,New York,36009.0,5728,92.0
1849,Cayuga,New York,36011.0,6357,92.0
1850,Chautauqua,New York,36013.0,8966,145.0
1851,Chemung,New York,36015.0,7787,136.0
1852,Chenango,New York,36017.0,3516,77.0
1853,Clinton,New York,36019.0,4856,33.0
1854,Columbia,New York,36021.0,4083,104.0


In [73]:
#create dictionary with values for 5 NYC counties
nyc = [{'county': 'New York County', 'state': 'New York', 'fips': 36061, 'cases': 139069, 'deaths': 3038},
       {'county': 'Bronx', 'state': 'New York', 'fips': 36005, 'cases': 177363, 'deaths': 4568},
       {'county': 'Kings', 'state': 'New York', 'fips': 36047, 'cases': 276971, 'deaths': 7326},
       {'county': 'Queens', 'state': 'New York', 'fips': 36081, 'cases': 273286, 'deaths': 6961},
       {'county': 'Richmond', 'state': 'New York', 'fips': 36085, 'cases': 75081, 'deaths': 1163},
      ]

In [77]:
#append dictionary to df
nyt_cases_deaths = nyt_cases_deaths.append(nyc, ignore_index=True)
nyt_cases_deaths

,county,state,fips,cases,deaths
0,Autauga,Alabama,1001.0,7262,113.0
1,Baldwin,Alabama,1003.0,22043,315.0
2,Barbour,Alabama,1005.0,2347,60.0
3,Bibb,Alabama,1007.0,2693,64.0
4,Blount,Alabama,1009.0,6988,139.0
...,...,...,...,...,...
3246,New York County,New York,36061.0,139069,3038.0
3247,Bronx,New York,36005.0,177363,4568.0
3248,Kings,New York,36047.0,276971,7326.0
3249,Queens,New York,36081.0,273286,6961.0


In [78]:
#validate
nyt_cases_deaths[nyt_cases_deaths['state'] == 'New York']

,county,state,fips,cases,deaths
1845,Albany,New York,36001.0,24740,357.0
1846,Allegany,New York,36003.0,3562,87.0
1847,Broome,New York,36007.0,18669,366.0
1848,Cattaraugus,New York,36009.0,5728,92.0
1849,Cayuga,New York,36011.0,6357,92.0
...,...,...,...,...,...
3246,New York County,New York,36061.0,139069,3038.0
3247,Bronx,New York,36005.0,177363,4568.0
3248,Kings,New York,36047.0,276971,7326.0
3249,Queens,New York,36081.0,273286,6961.0


In [79]:
#locate new york city row
nyt_cases_deaths.iloc[1873]

county    New York City
state          New York
fips              36061
cases            956795
deaths            33438
Name: 1873, dtype: object

In [80]:
#drop new york city row from df
nyt_cases_deaths.drop(axis=0, index=1873, inplace=True)

In [82]:
nyt_cases_deaths[nyt_cases_deaths['fips'].isna()]

,county,state,fips,cases,deaths
71,Bristol Bay plus Lake and Peninsula,Alaska,NaN,201,0.0
90,Unknown,Alaska,NaN,299,0.0
93,Yakutat plus Hoonah-Angoon,Alaska,NaN,81,2.0
180,Unknown,Arkansas,NaN,2967,0.0
315,Unknown,Connecticut,NaN,1179,1.0
320,Unknown,Delaware,NaN,227,0.0
385,Unknown,Florida,NaN,56,1012.0
534,Unknown,Georgia,NaN,3041,4.0
550,Unknown,Guam,NaN,9363,141.0
691,Unknown,Illinois,NaN,0,432.0


Kansas City is also consequential. KC spans 4 Missouri counties: Cass, Clay, Jackson, and Platte. In the absence of reliable county-level data that includes Kansas City numbers, I will split the cases across counties based on the approximate portion of the city located in each county:

- 50% attributed to Jackson County
- 30% attributed to Clay County
- 20% attributed to Platte County
- 0% attributed to Cass County

<img src='./images/kcmo.jpg' align center>

<center> Kansas City, MO Counties <br> <i> Source: <a href="https://www.marc.org/Community/Public-Health/Local-Public-Health-Agencies/MOHAKCA-Committee">Metropolitan Official Health Agencies of the Kansas City Area</a></i></center>

In [85]:
nyt_cases_deaths[(nyt_cases_deaths['county'] == 'Kansas City') & (nyt_cases_deaths['state'] == 'Missouri')]

,county,state,fips,cases,deaths
1547,Kansas City,Missouri,NaN,45765,597.0


In [87]:
kc_cases = nyt_cases_deaths.at[1547, 'cases']
kc_deaths = nyt_cases_deaths.at[1547, 'deaths']

In [91]:
nyt_cases_deaths[(nyt_cases_deaths.county == 'Jackson') & (nyt_cases_deaths.state=='Missouri')]

,county,state,fips,cases,deaths
1542,Jackson,Missouri,29095.0,40008,481.0


In [93]:
nyt_cases_deaths.at[1542, 'cases'] = nyt_cases_deaths.at[1542, 'cases'] + (kc_cases*.5)
nyt_cases_deaths.at[1542, 'deaths'] = nyt_cases_deaths.at[1542, 'deaths'] + (kc_deaths*.5)

nyt_cases_deaths[(nyt_cases_deaths.county == 'Jackson') & (nyt_cases_deaths.state=='Missouri')]

,county,state,fips,cases,deaths
1542,Jackson,Missouri,29095.0,85772,779.5


In [94]:
nyt_cases_deaths[(nyt_cases_deaths.county == 'Clay') & (nyt_cases_deaths.state=='Missouri')]

,county,state,fips,cases,deaths
1518,Clay,Missouri,29047.0,10559,174.0


In [96]:
nyt_cases_deaths.at[1518, 'cases'] = nyt_cases_deaths.at[1518, 'cases'] + (kc_cases*.3)
nyt_cases_deaths.at[1518, 'deaths'] = nyt_cases_deaths.at[1518, 'deaths'] + (kc_deaths*.3)

nyt_cases_deaths[(nyt_cases_deaths.county == 'Clay') & (nyt_cases_deaths.state=='Missouri')]

,county,state,fips,cases,deaths
1518,Clay,Missouri,29047.0,38017,532.2


In [97]:
nyt_cases_deaths[(nyt_cases_deaths.county == 'Platte') & (nyt_cases_deaths.state=='Missouri')]

,county,state,fips,cases,deaths
1579,Platte,Missouri,29165.0,4315,50.0


In [98]:
nyt_cases_deaths.at[1579, 'cases'] = nyt_cases_deaths.at[1579, 'cases'] + (kc_cases*.2)
nyt_cases_deaths.at[1579, 'deaths'] = nyt_cases_deaths.at[1579, 'deaths'] + (kc_deaths*.2)

nyt_cases_deaths[(nyt_cases_deaths.county == 'Platte') & (nyt_cases_deaths.state=='Missouri')]

,county,state,fips,cases,deaths
1579,Platte,Missouri,29165.0,13468,169.4


In [99]:
nyt_cases_deaths.drop(axis=0, index=1547, inplace=True)

In [101]:
nyt_cases_deaths[nyt_cases_deaths['fips'].isna()]

,county,state,fips,cases,deaths
71,Bristol Bay plus Lake and Peninsula,Alaska,NaN,201,0.0
90,Unknown,Alaska,NaN,299,0.0
93,Yakutat plus Hoonah-Angoon,Alaska,NaN,81,2.0
180,Unknown,Arkansas,NaN,2967,0.0
315,Unknown,Connecticut,NaN,1179,1.0
320,Unknown,Delaware,NaN,227,0.0
385,Unknown,Florida,NaN,56,1012.0
534,Unknown,Georgia,NaN,3041,4.0
550,Unknown,Guam,NaN,9363,141.0
691,Unknown,Illinois,NaN,0,432.0


Joplin, MO similarly sits within Jasper and Newton counties. 75% of Joplin cases/deats will be attributed to Jasper County and 20% attributed to Newton County.

<img src='./images/joplin_mo.png' align center>

<center> Joplin, MO Counties <br> <i> Source: <a href="https://en.wikipedia.org/wiki/Joplin,_Missouri">Arkayan Wikipedia user on Joplin, MO Wikipedia page</a></i></center>

In [102]:
nyt_cases_deaths[(nyt_cases_deaths.county == 'Joplin') & (nyt_cases_deaths.state=='Missouri')]

,county,state,fips,cases,deaths
1546,Joplin,Missouri,NaN,6789,112.0


In [103]:
nyt_cases_deaths[(nyt_cases_deaths.county == 'Jasper') & (nyt_cases_deaths.state=='Missouri')]

,county,state,fips,cases,deaths
1543,Jasper,Missouri,29097.0,10319,135.0


In [105]:
joplin_cases = nyt_cases_deaths.iloc[1546]['cases']
joplin_deaths = nyt_cases_deaths.iloc[1546]['deaths']

In [106]:
nyt_cases_deaths.at[1543, 'cases'] = nyt_cases_deaths.at[1543, 'cases'] + (joplin_cases*.75)
nyt_cases_deaths.at[1543, 'deaths'] = nyt_cases_deaths.at[1543, 'deaths'] + (joplin_deaths*.75)

nyt_cases_deaths[(nyt_cases_deaths.county == 'Jasper') & (nyt_cases_deaths.state=='Missouri')]

,county,state,fips,cases,deaths
1543,Jasper,Missouri,29097.0,15410,219.0


In [107]:
nyt_cases_deaths[(nyt_cases_deaths.county == 'Newton') & (nyt_cases_deaths.state=='Missouri')]

,county,state,fips,cases,deaths
1569,Newton,Missouri,29145.0,5939,80.0


In [108]:
nyt_cases_deaths.at[1569, 'cases'] = nyt_cases_deaths.at[1569, 'cases'] + (joplin_cases*.25)
nyt_cases_deaths.at[1569, 'deaths'] = nyt_cases_deaths.at[1569, 'deaths'] + (joplin_deaths*.25)

nyt_cases_deaths[(nyt_cases_deaths.county == 'Newton') & (nyt_cases_deaths.state=='Missouri')]

,county,state,fips,cases,deaths
1569,Newton,Missouri,29145.0,7636,108.0


In [110]:
nyt_cases_deaths.drop(axis=0, index=1546, inplace=True)

In [111]:
nyt_cases_deaths[nyt_cases_deaths['fips'].isna()]

,county,state,fips,cases,deaths
71,Bristol Bay plus Lake and Peninsula,Alaska,NaN,201,0.0
90,Unknown,Alaska,NaN,299,0.0
93,Yakutat plus Hoonah-Angoon,Alaska,NaN,81,2.0
180,Unknown,Arkansas,NaN,2967,0.0
315,Unknown,Connecticut,NaN,1179,1.0
320,Unknown,Delaware,NaN,227,0.0
385,Unknown,Florida,NaN,56,1012.0
534,Unknown,Georgia,NaN,3041,4.0
550,Unknown,Guam,NaN,9363,141.0
691,Unknown,Illinois,NaN,0,432.0


For this report, we'll focus on the 50 states and drop Guam and Puerto Rico.

In [112]:
nyt_cases_deaths.drop(axis=0, index=550, inplace=True)
nyt_cases_deaths.drop(axis=0, index=2400, inplace=True)

Next I'll explore how Alaska is represented elsewhere to determine how to incorporate counts from Bristol Bay plus Lake and Peninsula and Yakutat plus Hoonah-Angoon.

In [119]:
chr_df[ chr_df['State Abbreviation'] == 'AK']

,State Abbreviation,Name,Premature death raw value,Poor or fair health raw value,Poor physical health days raw value,Poor mental health days raw value,Adult smoking raw value,Adult obesity raw value,Physical inactivity raw value,Excessive drinking raw value,Preventable hospital stays raw value,Flu vaccinations raw value,Life expectancy raw value,Uninsured raw value,Ratio of population to primary care physicians.,Food environment index raw value,Food insecurity raw value,Homeownership raw value,Broadband access raw value,Unemployment raw value,Income inequality raw value,Median household income raw value,Air pollution - particulate matter raw value,Drinking water violations raw value,% Rural raw value
5-digit FIPS Code,,,,,,,,,,,,,,,,,,,,,,,,,
2016,AK,Aleutians West Census Area,3683.957514,0.175367,3.879493,3.523178,0.191869,0.423,0.269,0.165936,NaN,NaN,86.028507,0.176322,2861.500000,7.8,0.113,0.282224,0.771693,0.035774,3.016327,84726.0,NaN,1.0,1.000000
2020,AK,Anchorage Borough,7599.105226,0.144434,3.745846,3.923529,0.169737,0.302,0.169,0.178746,2758.0,0.40,79.305163,0.120902,872.868263,8.1,0.115,0.613389,0.899688,0.050514,3.862881,82512.0,6.4,1.0,0.041172
2050,AK,Bethel Census Area,15220.487811,0.322618,6.835134,5.874776,0.350080,0.341,0.277,0.165949,5053.0,0.07,71.590551,0.179462,958.736842,2.9,0.211,0.581644,0.732903,0.128826,5.697356,48262.0,NaN,1.0,0.739376
2068,AK,Denali Borough,4956.802990,0.144545,3.901392,3.998769,0.185680,0.234,0.210,0.198853,NaN,NaN,79.774076,0.121107,NaN,5.8,0.149,0.815661,0.898858,0.084778,3.107223,78434.0,NaN,0.0,1.000000
2070,AK,Dillingham Census Area,14346.400603,0.260209,5.916087,5.337797,0.301827,0.309,0.255,0.191435,3661.0,0.19,71.850674,0.187403,384.615385,4.8,0.161,0.620182,0.758935,0.081011,4.477378,57436.0,NaN,0.0,1.000000
2090,AK,Fairbanks North Star Borough,6820.353852,0.140356,3.963047,4.081826,0.174114,0.310,0.182,0.186492,2002.0,0.36,80.468359,0.123287,1221.864198,7.9,0.115,0.589201,0.857798,0.053839,3.724222,74076.0,13.9,0.0,0.308708
2100,AK,Haines Borough,3987.942101,0.133895,3.746857,4.096831,0.179213,0.291,0.194,0.218797,NaN,0.35,87.466206,0.139306,353.428571,7.5,0.133,0.664350,0.838133,0.089109,5.200470,62548.0,NaN,0.0,1.000000
2105,AK,Hoonah-Angoon Census Area,6075.647828,0.189877,4.558435,4.620964,0.236491,0.304,0.166,0.210931,NaN,NaN,84.210995,0.212613,NaN,4.3,0.155,0.754741,0.640961,0.113793,3.963815,53141.0,NaN,0.0,NaN
2110,AK,Juneau Borough,7387.024892,0.123552,3.584964,3.902636,0.159238,0.264,0.177,0.212131,2330.0,0.46,81.050335,0.108828,823.410256,8.2,0.104,0.647838,0.906674,0.044473,3.506579,85419.0,6.5,1.0,0.215444


I'll combine the "Yakutat plus Hoonah-Angoon" counts into Hoonah-Angoon Census Area	fips 2105, and "Bristol Bay plus Lake and Peninsula" into Lake and Peninsula Borough fips 2164.

In [121]:
nyt_cases_deaths[nyt_cases_deaths.state == "Alaska"]

,county,state,fips,cases,deaths
67,Aleutians East Borough,Alaska,2013.0,366,3.0
68,Aleutians West Census Area,Alaska,2016.0,727,0.0
69,Anchorage,Alaska,2020.0,31043,176.0
70,Bethel Census Area,Alaska,2050.0,3919,22.0
71,Bristol Bay plus Lake and Peninsula,Alaska,NaN,201,0.0
72,Denali Borough,Alaska,2068.0,111,0.0
73,Dillingham Census Area,Alaska,2070.0,233,2.0
74,Fairbanks North Star Borough,Alaska,2090.0,8004,44.0
75,Haines Borough,Alaska,2100.0,28,0.0
76,Juneau City and Borough,Alaska,2110.0,1470,3.0


In [ ]:
nyt_cases_deaths.at[71, 'fips'] = 2164
nyt_cases_deaths.at

From "Methodology and Definitions" section of The New York Times README on "Unknown" counties:

>Many state health departments choose to report cases separately when the patient’s county of residence is unknown or pending determination. In these instances, we record the county name as “Unknown.” As more information about these cases becomes available, the cumulative number of cases in “Unknown” counties may fluctuate.



In [114]:
nyt_cases_deaths[nyt_cases_deaths['state']== "Massachusetts"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 1226 to 1240
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   county  15 non-null     object 
 1   state   15 non-null     object 
 2   fips    14 non-null     float64
 3   cases   15 non-null     int64  
 4   deaths  15 non-null     float64
dtypes: float64(2), int64(1), object(2)
memory usage: 720.0+ bytes


# Appendix